https://github.com/hugosal/CirclesIntersections/blob/main/R/Librino_N.R

https://planetcalc.ru/8098/

In [ ]:
import numpy as np
from scipy.spatial import distance
from scipy.special import comb

In [ ]:
def two_circles_inters_points(cx, cy, r, distance):
    """Возвращает точки пересечения двух окружностей."""
    if distance >= r[0] + r[1] or distance <= abs(r[0] - r[1]) or distance == 0:
        return np.array([])  # Нет точек пересечения

    a = (r[0]**2 - r[1]**2 + distance**2) / (2 * distance)
    h = np.sqrt(r[0]**2 - a**2)
    x2 = cx[0] + a * (cx[1] - cx[0]) / distance
    y2 = cy[0] + a * (cy[1] - cy[0]) / distance
    x3 = x2 + h * (cy[1] - cy[0]) / distance
    y3 = y2 - h * (cx[1] - cx[0]) / distance
    x4 = x2 - h * (cy[1] - cy[0]) / distance
    y4 = y2 + h * (cx[1] - cx[0]) / distance
    return np.array([[x3, y3], [x4, y4]])

In [ ]:
def intersection_two_circles(cx, cy, r):
    """Возвращает площадь пересечения двух окружностей."""
    d = np.sqrt((cx[1] - cx[0])**2 + (cy[1] - cy[0])**2)

    if d >= r[0] + r[1]:
        return 0  # Нет пересечения
    if d <= abs(r[0] - r[1]):
        return np.pi * min(r[0], r[1])**2  # Одна окружность внутри другой

    r1_sq = r[0]**2
    r2_sq = r[1]**2

    part1 = r1_sq * np.arccos((d**2 + r1_sq - r2_sq) / (2 * d * r[0]))
    part2 = r2_sq * np.arccos((d**2 + r2_sq - r1_sq) / (2 * d * r[1]))
    part3 = 0.5 * np.sqrt((-d + r[0] + r[1]) * (d + r[0] - r[1]) * (d - r[0] + r[1]) * (d + r[0] + r[1]))

    return part1 + part2 - part3

In [ ]:
x1, y1, r1, x2, y2, r2, x3, y3, r3 = (2, 2, 6, 1, 1, 6, 9, 2, 4)

radii = [6,6,4]
centers_x = [2,1,9]
centers_y = [2,1,2]

indices = np.argsort(radii)[::-1]
print(indices)
cx = np.array(centers_x)[indices]
cy = np.array(centers_y)[indices]
r = np.array(radii)[indices]

Dist = np.linalg.norm(np.array([[cx[0], cy[0]], [cx[1], cy[1]], [cx[2], cy[2]]]), axis=1)
Dist = np.sqrt(((cx[:, None] - cx[None, :])**2 + (cy[:, None] - cy[None, :])**2))

print(Dist)

[1 0 2]
[[0.         1.41421356 8.06225775]
 [1.41421356 0.         7.        ]
 [8.06225775 7.         0.        ]]


In [ ]:
radii = [6,6,4]
indices = np.argsort(radii)[::-1]
print(indices)
np.array(radii)[indices]

[1 0 2]


array([6, 6, 4])

In [ ]:
np.array([[cx[0], cy[0]], [cx[1], cy[1]], [cx[2], cy[2]]])

array([[1, 1],
       [2, 2],
       [9, 2]])

представляет собой матрицу расстояний между центрами окружностей. Это двумерный массив, где элемент Dist[i, j] хранит расстояние между центрами окружностей i и j. Эта матрица используется для проверки пересечений окружностей и для вычисления площади пересечения.

In [ ]:
if np.any([Dist[i, j] > r[i] + r[j] for i, j in [(0, 1), (1, 2), (0, 2)]]):
    print('No interseption')

In [ ]:
inside = [r[i] >= Dist[i, j] + r[j] for i, j in [(0, 1), (1, 2), (0, 2)]]
if any(inside):
    print("Circle inside")
    max_container = np.argmin([i if x else float('inf') for i, x in enumerate(inside)])
    circle_most_inside = np.argmax([i if x else -1 for i, x in enumerate(inside)])
    other_circle = {0, 1, 2} - {max_container, circle_most_inside}
    # return intersection_two_circles([cx[circle_most_inside], cx[other_circle]],
    #                                 [cy[circle_most_inside], cy[other_circle]],
    #                                 [r[circle_most_inside], r[other_circle]])

In [ ]:
# Проверка, содержатся ли точки пересечения двух окружностей в третьей окружности
intersec_points_contained = [False, False, False]
for g in range(3):
    c1, c2 = list({0, 1, 2} - {g})
    D_c1_c2 = Dist[c1, c2]
    intsrs_points = two_circles_inters_points([cx[c1], cx[c2]], [cy[c1], cy[c2]], [r[c1], r[c2]], D_c1_c2)

    if intsrs_points.size == 0:
        continue

    points = np.vstack([np.array([cx[g], cy[g]]), intsrs_points])
    dist_matrix = np.linalg.norm(points - np.array([cx[g], cy[g]]), axis=1)

    if np.all(dist_matrix < r[g]):
        intersec_points_contained[g] = True

intersec_points_contained

[False, True, False]

In [ ]:
intersec_points_contained = [False, False, False]

# Compute points of overlap and check containment
for g in range(3):
    c1, c2 = {0, 1, 2} - {g}
    D_c1_c2 = Dist[c1, c2]

    intsrs_points = two_circles_inters_points([cx[c1], cx[c2]], [cy[c1], cy[c2]], [r[c1], r[c2]], D_c1_c2)
    print((cx[c1], cy[c1]), r[c1], '|', (cx[c2], cy[c2]), r[c2])
    print(intsrs_points)
    # матрица расстояний между центром окружности и точками пересечения
    dist_matrix = np.linalg.norm(intsrs_points - np.array([cx[g], cy[g]]), axis=1)
    print(dist_matrix)
    # Проверка, содержатся ли все точки пересечения внутри текущей окружности
    if np.all(dist_matrix < r[g]):
        intersec_points_contained[g] = True

    # dist_matrix = np.linalg.norm(np.array([cx[g], intsrs_points[:, 0]]), axis=1)
    # if np.all(dist_matrix < r[g]):
    #     intersec_points_contained[g] = True
    print('_'*10)

intersec_points_contained

(2, 2) 6 | (9, 2) 4
[[ 6.92857143 -1.42186845]
 [ 6.92857143  5.42186845]]
[6.40417098 7.39600431]
__________
(1, 1) 6 | (9, 2) 4
[[ 6.58620046 -1.18960371]
 [ 5.875338    4.49729601]]
[5.58630526 4.61028546]
__________
(1, 1) 6 | (2, 2) 6
[[ 5.71307489 -2.71307489]
 [-2.71307489  5.71307489]]
[ 5.7460379  12.28751596]
__________


[False, True, False]

In [ ]:
xx1 = 2
yy1 = 2
xx2 = 9
yy2 = 2
((xx1 - xx2) ** 2 + (yy1 - yy2) ** 2) ** 0.5

7.0

In [ ]:
xx1 = 9
yy1 = 2
xx2 = 6.92857143
yy2 = 6.92857143
((xx1 - xx2) ** 2 + (yy1 - yy2) ** 2) ** 0.5

5.346179258238961

In [ ]:
# Compute points of overlap and check containment
for g in range(3):
    c1, c2 = {0, 1, 2} - {g}
    D_c1_c2 = Dist[c1, c2]
    print(c1, c2, D_c1_c2)
    intsrs_points = two_circles_inters_points([cx[c1], cx[c2]], [cy[c1], cy[c2]], [r[c1], r[c2]], D_c1_c2)
    print(intsrs_points)
    print('-'*10)
    # dist_matrix = np.linalg.norm(np.array([cx[g], intsrs_points[:, 0]]), axis=1)
    # if np.all(dist_matrix < r[g]):
    #     intersec_points_contained[g] = True

1 2 7.0
[[ 6.92857143 -1.42186845]
 [ 6.92857143  5.42186845]]
----------
0 2 8.06225774829855
[[ 6.58620046 -1.18960371]
 [ 5.875338    4.49729601]]
----------
0 1 1.4142135623730951
[[ 5.71307489 -2.71307489]
 [-2.71307489  5.71307489]]
----------


# Отладка

In [ ]:
import numpy as np
from scipy.spatial import distance
from scipy.special import comb
from math import pi, sqrt, asin, acos, sin
import itertools

In [ ]:
# Вычисляет площадь пересечения двух кругов на основе их радиусов и расстояния между центрами
def intersection_two_circles(centers_x, centers_y, radii):
    r1, r2 = radii
    D = sqrt((centers_x[0] - centers_x[1])**2 + (centers_y[0] - centers_y[1])**2)

    if D <= abs(r1 - r2):
        return pi * min(r1, r2)**2

    if D > r1 + r2:
        return 0

    aa = (r1**2 - r2**2 + D**2) / (2 * D)
    bb = (r2**2 - r1**2 + D**2) / (2 * D)
    th1 = 2 * acos(aa / r1)
    th2 = 2 * acos(bb / r2)

    if th1 > pi:
        a1 = (r2**2 * (th2 + sin(2 * pi - th2))) / 2
    else:
        a1 = (r2**2 * (th2 - sin(th2))) / 2

    if th2 > pi:
        a2 = (r1**2 * (th1 + sin(2 * pi - th1))) / 2
    else:
        a2 = (r1**2 * (th1 - sin(th1))) / 2

    return a1 + a2

In [ ]:
radii = [6,6]
centers_x = [2,1]
centers_y = [2,1]

intersection_two_circles(centers_x, centers_y, radii)

96.16613874060447

In [ ]:
# Находит точки пересечения двух кругов, используя уравнения окружностей
def two_circles_inters_points(centers_x, centers_y, radii, distance):
    x1, x2 = centers_x
    y1, y2 = centers_y
    r1, r2 = radii

    a = (r1**2 - r2**2 + distance**2) / (2 * distance)
    h = sqrt(abs(r1**2 - a**2))

    x0 = x1 + a * (x2 - x1) / distance
    y0 = y1 + a * (y2 - y1) / distance

    intersection_x1 = x0 + h * (y2 - y1) / distance
    intersection_y1 = y0 - h * (x2 - x1) / distance

    intersection_x2 = x0 - h * (y2 - y1) / distance
    intersection_y2 = y0 + h * (x2 - x1) / distance

    return np.array([[intersection_x1, intersection_x2], [intersection_y1, intersection_y2]])

In [ ]:
radii = [6,6,4]
centers_x = [2,1,9]
centers_y = [2,1,2]
order_rad = np.argsort(radii)[::-1]
cx = np.array(centers_x)[order_rad]
cy = np.array(centers_y)[order_rad]
r = np.array(radii)[order_rad]
Dist = distance.cdist(list(zip(cx, cy)), list(zip(cx, cy)))
Dist

array([[0.        , 1.41421356, 8.06225775],
       [1.41421356, 0.        , 7.        ],
       [8.06225775, 7.        , 0.        ]])

In [ ]:
def intersection_three_circles(centers_x, centers_y, radii):
    # Circles must be sorted according to their radius
    order_rad = np.argsort(radii)[::-1]
    cx = np.array(centers_x)[order_rad]
    cy = np.array(centers_y)[order_rad]
    r = np.array(radii)[order_rad]
    Dist = distance.cdist(list(zip(cx, cy)), list(zip(cx, cy)))
    print('Params:')
    print(cx)
    print(cy)
    print(r)
    print("Distanse matrix computed:")
    print(Dist)
    # Check if any pair of circles are completely separate
    if any(Dist[i, j] > r[i] + r[j] for i in range(3) for j in range(i + 1, 3)):
        return 0
    else:
        print('Check if any pair of circles are completely separate: NO')

    # Check if any circle is completely inside another
    circles_inside = [r[i] >= Dist[i, j] + r[j] for i in range(3) for j in range(i + 1, 3)]
    print('Check if any circle is completely inside another', circles_inside)

    # List of intersecting circles
    circles_intersecting = [i for i in range(3) for j in range(i + 1, 3) if (r[i] - r[j]) < Dist[i, j] < (r[i] + r[j])]
    print("List of intersecting circles", circles_intersecting)

    print('any(circles_inside')
    if any(circles_inside):
        container_name = [1, 1, 2]
        contained_name = [2, 3, 3]

        maximum_container = min(np.array(container_name)[circles_inside])
        circle_most_inside = max(np.array(contained_name)[circles_inside])
        other_circle = [i for i in range(3) if i not in (maximum_container, circle_most_inside)]

        return intersection_two_circles(
            centers_x=[cx[circle_most_inside], cx[other_circle[0]]],
            centers_y=[cy[circle_most_inside], cy[other_circle[0]]],
            radii=[r[circle_most_inside], r[other_circle[0]]]
        )
    else:
        print("No any(circles_inside)")

    intersec_points_contained = [False, False, False]
    for g in range(3):
        c1, c2 = list({0, 1, 2} - {g})
        D_c1_c2 = Dist[c1, c2]
        intsrs_points = two_circles_inters_points([cx[c1], cx[c2]], [cy[c1], cy[c2]], [r[c1], r[c2]], D_c1_c2)

        if intsrs_points.size == 0:
            continue

        points = np.vstack([np.array([cx[g], cy[g]]), intsrs_points])
        dist_matrix = np.linalg.norm(points - np.array([cx[g], cy[g]]), axis=1)

        if np.all(dist_matrix < r[g]):
            intersec_points_contained[g] = True

    print('intersec_points_contained', intersec_points_contained)

    if sum(intersec_points_contained) == 2:
        print('here IF')
        containing_circle = [i for i in range(3) if not intersec_points_contained[i]][0]
        containing_circle_area = r[containing_circle]**2 * pi

        area_remove = 0
        for f in [i for i in range(3) if i != containing_circle]:
            area_inters = intersection_two_circles(
                centers_x=[cx[f], cx[containing_circle]],
                centers_y=[cy[f], cy[containing_circle]],
                radii=[r[f], r[containing_circle]]
            )
            area_remove += (containing_circle_area - area_inters)

        return containing_circle_area - area_remove

    # }else if (sum(intersec_points_contained) == 1){
    # print("HERE")
    # circs <- which(!intersec_points_contained)

    # result <- intersection_two_circles(centers_x = cx[circs], centers_y = cy[circs], radii =  r[circs])
    # }
    elif sum(intersec_points_contained) == 1:
        print("here ELIF")
        circs = [i for i in range(3) if not intersec_points_contained[i]]
        print(circs)
        print(intersection_two_circles([cx[circs[0]], cx[circs[1]]], [cy[circs[0]], cy[circs[1]]],[r[circs[0]], r[circs[1]]]))
        return intersection_two_circles([cx[circs[0]], cx[circs[1]]], [cy[circs[0]], cy[circs[1]]],[r[circs[0]], r[circs[1]]])
    # print('here')
    if not (r[0] - r[1] < Dist[0, 1] < (r[0] + r[1])):
        return 0

    rs = [r_i**2 for r_i in r]
    d = [Dist[0, 1], Dist[1, 2], Dist[0, 2]]

    x12 = (rs[0] - rs[1] + d[0]**2) / (2 * d[0])
    y12 = sqrt(((2 * d[0]**2) * (rs[0] + rs[1])) - (rs[0] - rs[1])**2 - d[0]**4) / (2 * d[0])

    costheta = (d[0]**2 + d[2]**2 - d[1]**2) / (2 * d[0] * d[2])
    sintheta = sqrt(1 - costheta**2)

    costhetap = -(d[0]**2 + d[1]**2 - d[2]**2) / (2 * d[0] * d[1])
    sinthetap = sqrt(1 - costhetap**2)

    if not ((x12 - (d[2] * costheta))**2 + (y12 - (d[2] * sintheta))**2 < rs[2]):
        return 0

    if not ((x12 - (d[2] * costheta))**2 + (y12 + (d[2] * sintheta))**2 > rs[2]):
        return 0

    x13p = (rs[0] - rs[2] + d[2]**2) / (2 * d[2])
    y13p = -sqrt((2 * d[2]**2 * (rs[0] + rs[2])) - (rs[0] - rs[2])**2 - d[2]**4) / (2 * d[2])

    x13 = (x13p * costheta) - (y13p * sintheta)
    y13 = (x13p * sintheta) + (y13p * costheta)

    x23pp = (rs[1] - rs[2] + d[1]**2) / (2 * d[1])
    y23pp = sqrt((2 * d[1]**2 * (rs[1] + rs[2])) - (rs[1] - rs[2])**2 - d[1]**4) / (2 * d[1])

    x23 = (x23pp * costhetap) - (y23pp * sinthetap) + d[0]
    y23 = (x23pp * sinthetap) + (y23pp * costhetap)

    if x23 - x13 == 0:
        return 0

    c1 = sqrt((x13 - x12)**2 + (y13 - y12)**2)
    c2 = sqrt((x12 - x23)**2 + (y12 - y23)**2)
    c3 = sqrt((x23 - x13)**2 + (y23 - y13)**2)

    A = (sqrt((c1 + c2 + c3) * (c2 + c3 - c1) * (c1 + c3 - c2) * (c1 + c2 - c3)) / 4) + \
        ((rs[0] * asin(c1 / (2 * r[0]))) - ((c1 / 4) * sqrt((4 * rs[0]) - c1**2))) + \
        ((rs[1] * asin(c2 / (2 * r[1]))) - ((c2 / 4) * sqrt((4 * rs[1]) - c2**2))) + \
        ((rs[2] * asin(c3 / (2 * r[2]))) - ((c3 / 4) * sqrt((4 * rs[2]) - c3**2)))

    return A

In [ ]:
radii = [6,6,4]
centers_x = [2,1,9]
centers_y = [2,1,2]
intersection_three_circles(centers_x, centers_y, radii)

OK
[0, 0, 1]
OK2
OK3
[False, True, False]
OK4


7.599910406248205

In [ ]:
import numpy as np
from scipy.spatial import distance
from scipy.special import factorial
import math
from itertools import combinations

def intersection_two_circles(centers_x, centers_y, radii):
    # Ваш код для вычисления площади пересечения двух кругов
    pass

def intersection_three_circles(centers_x, centers_y, radii):
    # Ваш код для вычисления площади пересечения трех кругов
    pass

def two_circles_inters_points(centers_x, centers_y, radii, distance):
    # Ваш код для вычисления точек пересечения двух кругов
    pass

def get_circle_number_from_binary(binary):
    return [i for i, bit in enumerate(binary) if bit == '1']

def get_transition_M_n_to_k(n, k, transitions):
    name_transition = f"{n + k - 1}:{n + k}"
    a_bar = transitions[name_transition]
    if k > 1:
        for i in range(1, k):
            name_transition = f"{n + k - i - 1}:{n + k - i}"
            a_bar = np.dot(a_bar, transitions[name_transition])
    return a_bar / factorial(k)

def get_sub_trellis(original, nvert, nelement, start):
    final_node = list(original[nvert][nelement].keys())[0]
    reduced_trellis = original[:nvert - 1]
    n_final_node = list(final_node)
    for back in range(nvert - 1, start - 1, -1):
        included_nodes = [x for x in reduced_trellis[back] if sum(np.array(list(x)) != np.array(n_final_node)) == (nvert - back)]
        reduced_trellis[back] = {k: reduced_trellis[back][k] for k in included_nodes}
    return reduced_trellis

def transition_from_a_i(a, start):
    transition_matrices_a = {}

    for r in range(start, len(a) - 1):
        next_lab = f"{r}:{r + 1}"

        # Проверка типа и содержимого
        if not isinstance(a[r + 1], (list, np.ndarray)):
            print(f"Error: Expected list or ndarray, but got {type(a[r + 1])}")
            continue
        if not isinstance(a[r], (list, np.ndarray)):
            print(f"Error: Expected list or ndarray, but got {type(a[r])}")
            continue

        names_rows = [list(name) for name in a[r + 1]]
        names_cols = [list(name) for name in a[r]]

        # Проверка содержимого
        print(f"names_rows: {names_rows}")
        print(f"names_cols: {names_cols}")

        # Создание матрицы переходов
        this_matrix = np.array([[sum(x != y) == 1 for y in names_cols] for x in names_rows])

        # Установка имен строк и столбцов
        if len(names_rows) > 0:
            row_names = [''.join(name) for name in names_rows]
            col_names = [''.join(name) for name in names_cols]
            this_matrix = pd.DataFrame(this_matrix, index=row_names, columns=col_names)

        transition_matrices_a[next_lab] = this_matrix

    return transition_matrices_a

def Librino_N(centers_x, centers_y, radii):
    N = len(centers_x)

    if len(centers_y) != N or len(radii) != N:
        raise ValueError("arguments must have the same length")

    if any(r <= 0 for r in radii):
        raise ValueError("radii must be greater than 0")

    areas = np.pi * np.array(radii) ** 2

    if N == 2:
        intersection = intersection_two_circles(centers_x, centers_y, radii)
        return {
            "1:2": intersection,
            "1": areas[0] - intersection,
            "2": areas[1] - intersection
        }

    d = distance.cdist(np.array([centers_x, centers_y]).T, np.array([centers_x, centers_y]).T)
    vector_dec_from_bin = [format(x, '0' + str(N) + 'b') for x in range(2 ** N)]
    sum_binary = np.array([bin(x).count('1') for x in range(2 ** N)])

    a_i = [sorted(np.where(sum_binary == x)[0], reverse=True) for x in range(1, N + 1)]

    transition_matrices = transition_from_a_i(a_i, start=1)

    a_i[0] = np.array([areas[int(''.join(x), 2)] for x in vector_dec_from_bin if sum(int(b) for b in x) == 1])
    a_i[1] = np.array([intersection_two_circles(
        [centers_x[i] for i in get_circle_number_from_binary(x)],
        [centers_y[i] for i in get_circle_number_from_binary(x)],
        [radii[i] for i in get_circle_number_from_binary(x)]
    ) for x in vector_dec_from_bin if sum(int(b) for b in x) == 2])
    a_i[2] = np.array([intersection_three_circles(
        [centers_x[i] for i in get_circle_number_from_binary(x)],
        [centers_y[i] for i in get_circle_number_from_binary(x)],
        [radii[i] for i in get_circle_number_from_binary(x)]
    ) for x in vector_dec_from_bin if sum(int(b) for b in x) == 3])

    if N >= 4:
        for u in range(4, N + 1):
            for v in range(len(a_i[u - 1])):
                another_one = u != 4
                minimum_depth = 1 if u == 4 else u - 2
                Abar = [None] * (u - 1 - another_one)
                subtrellis = get_sub_trellis(a_i, nvert=u, nelement=v, start=minimum_depth)
                transition_sub = transition_from_a_i(subtrellis, start=minimum_depth)

                if any(np.array(list(subtrellis[-1].values())) == 0):
                    a_i[u - 1][v] = 0
                    continue

                for e in range(minimum_depth, u - 1 - another_one):
                    product_sum = 0
                    if e < u - 1:
                        for j in range(e + 1, u - 1):
                            tnk = get_transition_M_n_to_k(n=e, k=j - e, transitions=transition_sub)
                            this_rep = (-1) ** (j - e + 1) * np.dot(tnk.T, np.array(list(subtrellis[j].values())))
                            product_sum += this_rep
                    Abar[e] = list(subtrellis[e].values()) - product_sum

                if u == 4:
                    agam = min(Abar[0])
                    bgam = max(-Abar[1])
                    cgam = min(Abar[2])

                    if any(np.array(list(subtrellis[2].values())) < 1e-6):
                        a_i[u - 1][v] = min(Abar[u - 2])
                    else:
                        if cgam > bgam and abs(cgam - cgam) < 1e-5:
                            four_circles = get_circle_number_from_binary(vector_dec_from_bin[v])
                            combinations_2 = list(combinations(four_circles, 2))

                            intersc_pts = []
                            for c1, c2 in combinations_2:
                                D_c1_c2 = d[c1, c2]
                                points = two_circles_inters_points(
                                    [centers_x[c1], centers_x[c2]],
                                    [centers_y[c1], centers_y[c2]],
                                    [radii[c1], radii[c2]],
                                    D_c1_c2
                                )
                                intersc_pts.append(points)

                            intersections_mat = np.vstack(intersc_pts)

                            inters_pts_inside_circle = []
                            for x in four_circles:
                                distances_to_inters = distance.cdist(
                                    np.array([[centers_x[x], centers_y[x]]]),
                                    intersections_mat
                                ).flatten()
                                not_including = [x not in y.split(':') for y in map(str, range(len(intersections_mat)))]
                                inside_pts = np.where((distances_to_inters < radii[x]) & not_including)[0]
                                inters_pts_inside_circle.append(inside_pts.tolist())

                            the_one = [False] * 4
                            for j in range(len(inters_pts_inside_circle)):
                                the_one[j] = all(len(set(inters_pts_inside_circle[j]) & set(inters_pts_inside_circle[x])) == 1
                                                 for x in range(len(inters_pts_inside_circle)) if x != j)

                            if all(len(x) == 3 for x in inters_pts_inside_circle):
                                if sum(the_one) == 1:
                                    a_i[u - 1][v] = min(Abar[u - 2])
                                else:
                                    a_i[u - 1][v] = max(-Abar[u - 3])
                            else:
                                a_i[u - 1][v] = max(-Abar[u - 3])
                        else:
                            a_i[u - 1][v] = max(-Abar[u - 3])
                else:
                    a_i[u - 1][v] = max(-Abar[u - 3])

    Intersections_final = []
    for i in range(N):
        This_intersection_area = a_i[i]
        product_sum = 0
        if i < N - 1:
            product_sum = np.sum([
                (-1) ** (j - i + 1) * np.dot(get_transition_M_n_to_k(n=i, k=j - i, transitions=transition_matrices).T, a_i[j])
                for j in range(i + 1, N)
            ], axis=0)
        Intersections_final.append(This_intersection_area - product_sum)

    for l in range(len(Intersections_final) - 1, -1, -1):
        this_vect = Intersections_final[l]
        if len(this_vect) == 0:
            Intersections_final[l] = None
        else:
            Intersections_final[l] = this_vect[:, 0]

    return Intersections_final

In [ ]:
radii = [6,6,4]
centers_x = [2,1,9]
centers_y = [2,1,2]
Librino_N(centers_x, centers_y, radii)

TypeError: 'numpy.int64' object is not iterable